## 1、 IntermediateLayerGetter

In [1]:
import torch
import torchvision.models as models
from torchvision.models._utils import IntermediateLayerGetter

model = models.resnet50()

# 查看网络中间层的层名
for name, layer in model.named_children():
    print(name)
print('--------------------------------')

# 指定需要哪些层的输出， 只能指定一级子层名称，无法指定二级子层名称
return_layers = {'layer3': "out1", 'layer4': "out2",}

# 生成模型对象
new_model = IntermediateLayerGetter(model, return_layers=return_layers)

# 调用forward方法，得到我们要的中间层的输出
output = new_model(torch.rand(1, 3, 224, 224))

print(output.keys())
print(output["out1"].shape)
print(output["out2"].shape)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
--------------------------------
odict_keys(['out1', 'out2'])
torch.Size([1, 1024, 14, 14])
torch.Size([1, 2048, 7, 7])




## 2、 register_forward_hook


### 例1

In [2]:
import torch
import torch.nn as nn

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(10, 5)
        self.layer2 = nn.ReLU()
        self.layer3 = nn.Linear(5, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

# 创建模型实例
model = SimpleModel()

# 定义一个用于处理中间层输出的回调函数
features = []
def forward_hook(module, input, output):
    features.append(output.clone().detach())

# 注册 forward hook 到指定层
hook_layer = model.layer2
hook_handle = hook_layer.register_forward_hook(forward_hook)

# 创建输入数据
input_data = torch.randn(3, 10)

# 模型前向传播
output = model(input_data)

# 注销 forward hook
hook_handle.remove()

print(output.shape)
print(features[0].shape)

torch.Size([3, 2])
torch.Size([3, 5])




### 例2 ： 获取 resnet-50 网络中 model.layer3[1].conv2  这一层的输出

In [3]:
import torch
import torchvision.models as models

model = models.resnet50()

# 注册 forward hook 到指定层
hook_layer = model.layer3[1].conv2

# # 查看网络中间层的层名
# for name, layer in model.named_modules():
#     print(name)
# print('--------------------------------')

# 定义一个用于处理中间层输出的回调函数
features = []
def forward_hook(module, input, output):
    features.append(output.clone().detach())


hook_handle = hook_layer.register_forward_hook(forward_hook)

# 创建输入数据
input_data = torch.randn(1, 3, 224, 224)

# 模型前向传播
output = model(input_data)

# 注销 forward hook
hook_handle.remove()

print(output.shape)
print(features[0].shape)

torch.Size([1, 1000])
torch.Size([1, 256, 14, 14])
